# Data Profiling and Validation

## Objective
Comprehensive data understanding and quality assessment of the Ames Housing dataset to establish baseline characteristics and validate data integrity for subsequent analysis.

## Contents
1. Data Loading and Initial Assessment
2. Automated Data Profiling
3. Statistical Validation
4. Data Type Optimization
5. Missing Value Analysis
6. Feature Distribution Analysis
7. Data Quality Summary


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## 1. Data Loading and Initial Assessment


In [2]:
# Load the dataset
df = pd.read_csv('../../data/raw/train.csv')

print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()


Dataset Shape: (1460, 81)

First 5 rows:


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Basic dataset information
print("Dataset Info:")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]:,}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Duplicated Rows: {df.duplicated().sum()}")

print("\nData Types Distribution:")
print(df.dtypes.value_counts())


Dataset Info:
Rows: 1,460
Columns: 81
Memory Usage: 3.43 MB
Duplicated Rows: 0

Data Types Distribution:
object     43
int64      35
float64     3
Name: count, dtype: int64


## 2. Automated Data Profiling


In [4]:
def data_profiling_report(dataframe):
    """Generate comprehensive data profiling report"""
    
    profile = {
        'column': [],
        'dtype': [],
        'non_null_count': [],
        'null_count': [],
        'null_percentage': [],
        'unique_count': [],
        'unique_percentage': [],
        'most_frequent_value': [],
        'frequency_of_most_frequent': []
    }
    
    for col in dataframe.columns:
        profile['column'].append(col)
        profile['dtype'].append(str(dataframe[col].dtype))
        profile['non_null_count'].append(dataframe[col].count())
        profile['null_count'].append(dataframe[col].isnull().sum())
        profile['null_percentage'].append(round(dataframe[col].isnull().mean() * 100, 2))
        profile['unique_count'].append(dataframe[col].nunique())
        profile['unique_percentage'].append(round(dataframe[col].nunique() / len(dataframe) * 100, 2))
        
        if dataframe[col].count() > 0:
            most_frequent = dataframe[col].value_counts().index[0]
            frequency = dataframe[col].value_counts().iloc[0]
            profile['most_frequent_value'].append(most_frequent)
            profile['frequency_of_most_frequent'].append(frequency)
        else:
            profile['most_frequent_value'].append(None)
            profile['frequency_of_most_frequent'].append(0)
    
    return pd.DataFrame(profile)

# Generate profiling report
profile_df = data_profiling_report(df)
print("Data Profiling Report:")
profile_df


Data Profiling Report:


,column,dtype,non_null_count,null_count,null_percentage,unique_count,unique_percentage,most_frequent_value,frequency_of_most_frequent
0,Id,int64,1460,0,0.00,1460,100.00,1,1
1,MSSubClass,int64,1460,0,0.00,15,1.03,20,536
2,MSZoning,object,1460,0,0.00,5,0.34,RL,1151
3,LotFrontage,float64,1201,259,17.74,110,7.53,60.0,143
4,LotArea,int64,1460,0,0.00,1073,73.49,7200,25
5,Street,object,1460,0,0.00,2,0.14,Pave,1454
6,Alley,object,91,1369,93.77,2,0.14,Grvl,50
7,LotShape,object,1460,0,0.00,4,0.27,Reg,925
8,LandContour,object,1460,0,0.00,4,0.27,Lvl,1311
9,Utilities,object,1460,0,0.00,2,0.14,AllPub,1459


In [ ]:
# Identify columns with missing values
missing_data = profile_df[profile_df['null_percentage'] > 0].sort_values('null_percentage', ascending=False)
print(f"Columns with missing values: {len(missing_data)}")
if len(missing_data) > 0:
    print("\nTop 10 columns with highest missing percentage:")
    print(missing_data[['column', 'null_count', 'null_percentage']].head(10))


In [ ]:
# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"Numeric columns: {len(numeric_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

# Statistical summary for numeric columns
print("\nNumeric Columns Statistical Summary:")
df[numeric_cols].describe()


In [ ]:
# Outlier detection using IQR method
def detect_outliers_iqr(dataframe, columns):
    """Detect outliers using Interquartile Range method"""
    outlier_summary = []
    
    for col in columns:
        Q1 = dataframe[col].quantile(0.25)
        Q3 = dataframe[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = dataframe[(dataframe[col] < lower_bound) | (dataframe[col] > upper_bound)][col]
        outlier_count = len(outliers)
        outlier_percentage = (outlier_count / len(dataframe)) * 100
        
        outlier_summary.append({
            'column': col,
            'outlier_count': outlier_count,
            'outlier_percentage': round(outlier_percentage, 2),
            'lower_bound': round(lower_bound, 2),
            'upper_bound': round(upper_bound, 2)
        })
    
    return pd.DataFrame(outlier_summary)

# Detect outliers in numeric columns
outlier_df = detect_outliers_iqr(df, numeric_cols)
outlier_df_filtered = outlier_df[outlier_df['outlier_count'] > 0].sort_values('outlier_percentage', ascending=False)

print("Outlier Analysis (IQR Method):")
print(f"Columns with outliers: {len(outlier_df_filtered)}")
if len(outlier_df_filtered) > 0:
    print("\nTop 10 columns with highest outlier percentage:")
    print(outlier_df_filtered[['column', 'outlier_count', 'outlier_percentage']].head(10))


In [ ]:
# Memory optimization function
def optimize_memory_usage(dataframe):
    """Optimize memory usage by converting data types"""
    original_memory = dataframe.memory_usage(deep=True).sum() / 1024**2
    
    optimized_df = dataframe.copy()
    
    for col in optimized_df.columns:
        col_type = optimized_df[col].dtype
        
        if col_type != 'object':
            c_min = optimized_df[col].min()
            c_max = optimized_df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    optimized_df[col] = optimized_df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    optimized_df[col] = optimized_df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    optimized_df[col] = optimized_df[col].astype(np.int32)
                    
            elif str(col_type)[:5] == 'float':
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    optimized_df[col] = optimized_df[col].astype(np.float32)
        
        else:
            # Convert object columns to category if cardinality is low
            unique_ratio = optimized_df[col].nunique() / len(optimized_df)
            if unique_ratio < 0.5:  # Less than 50% unique values
                optimized_df[col] = optimized_df[col].astype('category')
    
    optimized_memory = optimized_df.memory_usage(deep=True).sum() / 1024**2
    reduction = (original_memory - optimized_memory) / original_memory * 100
    
    print(f"Original memory usage: {original_memory:.2f} MB")
    print(f"Optimized memory usage: {optimized_memory:.2f} MB")
    print(f"Memory reduction: {reduction:.2f}%")
    
    return optimized_df

# Apply memory optimization
df_optimized = optimize_memory_usage(df)

print("\nData types after optimization:")
print(df_optimized.dtypes.value_counts())


In [ ]:
# Comprehensive missing value analysis
def missing_value_analysis(dataframe):
    """Analyze missing value patterns"""
    
    missing_df = pd.DataFrame({
        'column': dataframe.columns,
        'missing_count': dataframe.isnull().sum(),
        'missing_percentage': (dataframe.isnull().sum() / len(dataframe)) * 100,
        'data_type': dataframe.dtypes
    })
    
    missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_percentage', ascending=False)
    
    if len(missing_df) > 0:
        print("Missing Value Analysis:")
        print(missing_df)
        
        # Visualize missing values pattern
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        missing_df.plot(x='column', y='missing_percentage', kind='bar', ax=plt.gca())
        plt.title('Missing Value Percentage by Column')
        plt.xticks(rotation=45)
        plt.ylabel('Missing Percentage (%)')
        
        # Missing value heatmap for top columns
        if len(missing_df) > 0:
            top_missing_cols = missing_df.head(10)['column'].tolist()
            plt.subplot(1, 2, 2)
            sns.heatmap(dataframe[top_missing_cols].isnull(), cbar=True, yticklabels=False)
            plt.title('Missing Value Pattern (Top 10 Columns)')
            plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Missing value correlation
        if len(missing_df) > 1:
            missing_corr = dataframe[missing_df['column'].tolist()].isnull().corr()
            plt.figure(figsize=(10, 8))
            sns.heatmap(missing_corr, annot=True, cmap='coolwarm', center=0)
            plt.title('Missing Value Correlation Matrix')
            plt.show()
    
    else:
        print("No missing values found in the dataset!")
        
    return missing_df

# Perform missing value analysis
missing_analysis = missing_value_analysis(df)


In [ ]:
# Target variable analysis
target_col = 'SalePrice'
if target_col in df.columns:
    print("Target Variable Analysis (SalePrice):")
    print(f"Mean: ${df[target_col].mean():,.2f}")
    print(f"Median: ${df[target_col].median():,.2f}")
    print(f"Standard Deviation: ${df[target_col].std():,.2f}")
    print(f"Min: ${df[target_col].min():,.2f}")
    print(f"Max: ${df[target_col].max():,.2f}")
    print(f"Skewness: {df[target_col].skew():.3f}")
    print(f"Kurtosis: {df[target_col].kurtosis():.3f}")
    
    # Target distribution visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Histogram
    axes[0].hist(df[target_col], bins=50, alpha=0.7, color='skyblue')
    axes[0].set_title('SalePrice Distribution')
    axes[0].set_xlabel('Sale Price ($)')
    axes[0].set_ylabel('Frequency')
    
    # Box plot
    axes[1].boxplot(df[target_col])
    axes[1].set_title('SalePrice Box Plot')
    axes[1].set_ylabel('Sale Price ($)')
    
    # Log transformation
    axes[2].hist(np.log1p(df[target_col]), bins=50, alpha=0.7, color='lightgreen')
    axes[2].set_title('Log-Transformed SalePrice')
    axes[2].set_xlabel('Log(Sale Price + 1)')
    axes[2].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Distribution analysis for key numeric features
def analyze_numeric_distributions(dataframe, columns, max_plots=12):
    """Analyze distributions of numeric columns"""
    
    # Select top numeric columns by correlation with target (if available)
    if 'SalePrice' in dataframe.columns:
        correlations = dataframe[columns].corrwith(dataframe['SalePrice']).abs().sort_values(ascending=False)
        top_columns = correlations.head(max_plots).index.tolist()
    else:
        top_columns = columns[:max_plots]
    
    n_cols = 4
    n_rows = (len(top_columns) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, col in enumerate(top_columns):
        if i < len(axes):
            axes[i].hist(dataframe[col].dropna(), bins=30, alpha=0.7, color='steelblue')
            axes[i].set_title(f'{col}\nSkew: {dataframe[col].skew():.2f}')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Frequency')
    
    # Hide empty subplots
    for i in range(len(top_columns), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    return top_columns

# Analyze distributions
print("Key Numeric Feature Distributions:")
key_numeric_features = analyze_numeric_distributions(df, numeric_cols)


In [ ]:
# Categorical feature analysis
def analyze_categorical_features(dataframe, columns, max_categories=10):
    """Analyze categorical features"""
    
    categorical_summary = []
    
    for col in columns:
        value_counts = dataframe[col].value_counts()
        categorical_summary.append({
            'column': col,
            'unique_count': dataframe[col].nunique(),
            'most_frequent': value_counts.index[0] if len(value_counts) > 0 else None,
            'most_frequent_count': value_counts.iloc[0] if len(value_counts) > 0 else 0,
            'most_frequent_percentage': round(value_counts.iloc[0] / len(dataframe) * 100, 2) if len(value_counts) > 0 else 0
        })
    
    cat_df = pd.DataFrame(categorical_summary)
    print("Categorical Features Summary:")
    print(cat_df.head(10))
    
    # Visualize top categorical features
    high_cardinality = cat_df[cat_df['unique_count'] <= max_categories].head(8)
    
    if len(high_cardinality) > 0:
        n_cols = 2
        n_rows = (len(high_cardinality) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
        
        for i, row in high_cardinality.iterrows():
            col = row['column']
            if i < len(axes):
                value_counts = dataframe[col].value_counts().head(max_categories)
                axes[i].bar(range(len(value_counts)), value_counts.values, color='coral')
                axes[i].set_title(f'{col} (Unique: {row["unique_count"]})')
                axes[i].set_xticks(range(len(value_counts)))
                axes[i].set_xticklabels(value_counts.index, rotation=45, ha='right')
                axes[i].set_ylabel('Count')
        
        # Hide empty subplots
        for i in range(len(high_cardinality), len(axes)):
            if i < len(axes):
                axes[i].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    
    return cat_df

# Analyze categorical features
print("Categorical Feature Analysis:")
categorical_analysis = analyze_categorical_features(df, categorical_cols)


In [ ]:
# Comprehensive data quality summary
def generate_data_quality_summary(dataframe):
    """Generate comprehensive data quality summary"""
    
    summary = {
        'Dataset Overview': {
            'Total Rows': f"{dataframe.shape[0]:,}",
            'Total Columns': f"{dataframe.shape[1]:,}",
            'Memory Usage': f"{dataframe.memory_usage(deep=True).sum() / 1024**2:.2f} MB",
            'Duplicate Rows': f"{dataframe.duplicated().sum():,}"
        },
        
        'Data Types': {
            'Numeric Columns': f"{len(dataframe.select_dtypes(include=[np.number]).columns):,}",
            'Categorical Columns': f"{len(dataframe.select_dtypes(include=['object']).columns):,}",
            'Boolean Columns': f"{len(dataframe.select_dtypes(include=['bool']).columns):,}"
        },
        
        'Data Quality': {
            'Columns with Missing Values': f"{dataframe.isnull().any().sum():,}",
            'Total Missing Values': f"{dataframe.isnull().sum().sum():,}",
            'Missing Value Percentage': f"{(dataframe.isnull().sum().sum() / (dataframe.shape[0] * dataframe.shape[1]) * 100):.2f}%"
        }
    }
    
    # Outlier summary
    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns
    outlier_counts = []
    for col in numeric_cols:
        Q1 = dataframe[col].quantile(0.25)
        Q3 = dataframe[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = len(dataframe[(dataframe[col] < lower_bound) | (dataframe[col] > upper_bound)])
        outlier_counts.append(outliers)
    
    summary['Outlier Analysis'] = {
        'Columns with Outliers': f"{len([x for x in outlier_counts if x > 0]):,}",
        'Total Outliers': f"{sum(outlier_counts):,}",
        'Average Outliers per Column': f"{np.mean(outlier_counts):.1f}"
    }
    
    return summary

# Generate and display summary
quality_summary = generate_data_quality_summary(df)

print("=== DATA QUALITY SUMMARY ===\n")
for category, metrics in quality_summary.items():
    print(f"{category}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")
    print()


In [ ]:
# Key recommendations based on analysis
print("=== KEY RECOMMENDATIONS ===\n")

recommendations = []

# Missing value recommendations
missing_cols = df.isnull().sum()
missing_cols = missing_cols[missing_cols > 0]
if len(missing_cols) > 0:
    high_missing = missing_cols[missing_cols > len(df) * 0.5]
    if len(high_missing) > 0:
        recommendations.append(f"Consider dropping columns with >50% missing values: {list(high_missing.index)}")
    
    moderate_missing = missing_cols[(missing_cols > 0) & (missing_cols <= len(df) * 0.5)]
    if len(moderate_missing) > 0:
        recommendations.append(f"Apply imputation strategies for columns: {list(moderate_missing.index[:5])}")

# Outlier recommendations
outlier_df = detect_outliers_iqr(df, numeric_cols)
high_outlier_cols = outlier_df[outlier_df['outlier_percentage'] > 10]['column'].tolist()
if len(high_outlier_cols) > 0:
    recommendations.append(f"Investigate high outlier columns: {high_outlier_cols[:5]}")

# Skewness recommendations
skewed_cols = []
for col in numeric_cols:
    if abs(df[col].skew()) > 2:
        skewed_cols.append(col)
if len(skewed_cols) > 0:
    recommendations.append(f"Consider log transformation for highly skewed columns: {skewed_cols[:5]}")

# High cardinality categorical recommendations
high_cardinality_cats = []
for col in categorical_cols:
    if df[col].nunique() > len(df) * 0.8:
        high_cardinality_cats.append(col)
if len(high_cardinality_cats) > 0:
    recommendations.append(f"Review high cardinality categorical columns: {high_cardinality_cats}")

# Display recommendations
if recommendations:
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
else:
    print("Dataset appears to be in good condition. Proceed with feature engineering.")

print("\n=== DATA PROFILING COMPLETE ===")
print("Dataset ready for feature engineering and modeling phases.")
